## Imports

In [1]:
############
# Built-in #
############
import os

###############
# Third-party #
###############
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

## Database Connection Setup

In [2]:
HOST = 'localhost'
USER = 'db_user'
PORT = 5438
DATABASE = 'msd_db'
PASSWORD = 'LetMeIn'
TABLE_NAME = 'song_info'

# Connect with SQLAlchemy
conn_string = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
sqla_conn = create_engine(conn_string)

## Data Loading

In [3]:
tracks_df = pd.read_sql(
    f'''
        SELECT 
            *
        FROM {TABLE_NAME}
    ''',
    sqla_conn
)
tracks_df.head()

,count
0,10000


## Data Preprocessing